<a href="https://colab.research.google.com/github/CodeRoninSY/colab_generate_AI/blob/main/Generating_AI_Photos_Leap_AI_Realistic_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time

# get api key from Leap AI https://www.tryleap.ai
API_KEY = "de94877e-d732-447c-857b-de9cff3a95f2"

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

In [ ]:
def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 100,
        "width": 512,
        "height": 512,
        "numberOfImages": 1,
        "promptStrength": 8,
        "enhancePrompt": False,
        "restoreFaces": True,
        "upscaleBy": "x1"
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    print(data)
    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status


def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    images = None

    if len(data["images"]):
        images = data["images"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, images




In [ ]:
# modelID refers to Realistic Vision model. We can now use it to generate images from.
# For more models go here: https://docs.tryleap.ai/reference/pre-trained-models
model_id = "eab32df0-de26-4b83-a908-a83f3015e971"

inference_id, status = generate_image(
    model_id,
    prompt="A profile photo of Superman, wearing similar to Darth Vader \
    styled suit, \
    sunset background, just showing above torso, smiling, \
    generated by '90s anime style, at New York."
)

while status != "finished":
    time.sleep(10)
    inference_id, status, images = get_inference_job(model_id, inference_id)

for image in images:
    print("Image ready:", image["uri"])

{'status': 'queued', 'id': '0ff699c7-5844-49f1-9724-1d511758e1a3', 'prompt': "A profile photo of Superman, wearing similar to Darth Vader     styled suit,     sunset background, just showing above torso, smiling,     generated by '90s anime style, at New York.", 'negativePrompt': '', 'seed': 783245163, 'width': 512, 'height': 512, 'numberOfImages': 1, 'steps': 100, 'createdAt': '2023-06-24T16:12:22.836Z', 'promptStrength': 8, 'images': [], 'modelId': 'eab32df0-de26-4b83-a908-a83f3015e971', 'upscalingOption': 'x1', 'sampler': 'euler_a'}
Inference ID: 0ff699c7-5844-49f1-9724-1d511758e1a3. Status: queued
Inference ID: 0ff699c7-5844-49f1-9724-1d511758e1a3. State: processing
Inference ID: 0ff699c7-5844-49f1-9724-1d511758e1a3. State: processing
Inference ID: 0ff699c7-5844-49f1-9724-1d511758e1a3. State: finished
Image ready: https://static.tryleap.ai/image-gen-0ff699c7-5844-49f1-9724-1d511758e1a3/generated_images/0.png
